In [2]:
from os import path
import os
from pydub import AudioSegment
import pandas as pd

In [3]:
from pathlib import Path

ROOT_DIR = Path.cwd().parent
INPUT_DIR = ROOT_DIR / 'input'
DATA_DIR = INPUT_DIR / "bengaliai-speech"
TRAIN_MP3_DIR = DATA_DIR / "train_mp3s"
TRAIN_WAV_DIR = INPUT_DIR / "train_wavs"

if os.path.exists(TRAIN_WAV_DIR) == False:
    os.mkdir(TRAIN_WAV_DIR)

In [4]:
train_df = pd.read_csv(DATA_DIR / "train.csv")
train_df.head()

,id,sentence,split
0,000005f3362c,ও বলেছে আপনার ঠিকানা!,train
1,00001dddd002,কোন মহান রাষ্ট্রের নাগরিক হতে চাও?,train
2,00001e0bc131,"আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।",train
3,000024b3d810,নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন...,train
4,000028220ab3,"হুমম, ওহ হেই, দেখো।",train


In [5]:
# TRAIN_WAV = DATA_DIR / "train_wavs" の中にあるファイルの個数
import os
print(len(os.listdir(TRAIN_WAV_DIR)))

898877


In [6]:
# test_df の、"id" 列の .mp3 を削除
# test_df = pd.read_csv(DATA_DIR / "test.csv")
# test_df["id"] = test_df["id"].str.replace(".mp3", "")

In [7]:
train_ids = train_df['id'].tolist()
len(train_ids)

963636

In [8]:
from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count
cpu_count()

32

In [9]:
def process(train_id):
    src_path = TRAIN_MP3_DIR / f'{train_id}.mp3'
    dst_path = TRAIN_WAV_DIR / f'{train_id}.wav'
    sound = AudioSegment.from_mp3(src_path)
    # もし 4 秒以上あるようなら 4 秒までを wav に変換
    if len(sound) >= 4000:
        sound = sound[:4000]
    sound.export(dst_path, format="wav")

In [10]:
# from tqdm import tqdm
# _ = Parallel(n_jobs=cpu_count())(
#     delayed(process)(train_id)
#     for train_id in tqdm(train_ids)
# )

In [21]:
# TRAIN_MP3_DIR の中にあるファイルを一つ取り、os.path.getsize() でサイズを取得して print
import os
max_size = 0

for train_id in train_ids:
    src_path = TRAIN_MP3_DIR / f'{train_id}'
    size = os.path.getsize(src_path)
    if size > max_size:
        max_size = size

print(max_size)

145845


In [12]:
import os
example_file_size = os.path.getsize(DATA_DIR / "examples/Slang Profanity.mp3")
print(example_file_size)

485361


In [13]:
import os
train_ids = sorted(os.listdir(TRAIN_MP3_DIR))
train_wav_ids = sorted(os.listdir(TRAIN_WAV_DIR))

train_ids_without_extension = [train_id.split('.')[0] for train_id in train_ids]
train_wav_ids_without_extension = [train_id.split('.')[0] for train_id in train_wav_ids]

In [14]:
# train_wav_ids_without_extension_set = set(train_wav_ids_without_extension)

# for train_id in train_ids_without_extension:
#     if train_id not in train_wav_ids_without_extension_set:
#         process(train_id)

In [15]:
# TRAIN_WAV = DATA_DIR / "train_wavs" の中にあるファイルの個数
import os
print(len(os.listdir(TRAIN_WAV_DIR)))

898877


In [16]:
# TRAIN_WAV の中に、0バイトのファイルがないか確かめる
import os
from pathlib import Path

ROOT_DIR = Path.cwd().parent
INPUT_DIR = ROOT_DIR / 'input'
DATA_DIR = INPUT_DIR / "bengaliai-speech"
TRAIN_WAV_DIR = INPUT_DIR / "train_wavs"

train_wav_ids = sorted(os.listdir(TRAIN_WAV_DIR))
train_wav_ids_without_extension = [train_id.split('.')[0] for train_id in train_wav_ids]

# train_wav_ids_without_extension の中にあるファイルのサイズの平均を算出
total_size = 0
for train_wav_id in train_wav_ids_without_extension:
    file_path = TRAIN_WAV_DIR / f"{train_wav_id}.wav"
    total_size += os.path.getsize(file_path)

average_size = total_size / len(train_wav_ids_without_extension)
print(f"The average size of files in TRAIN_WAV_DIR is {average_size} bytes.")

# 空のファイルがあればそのファイル名を print
nearly_empty_files = []
for train_wav_id in train_wav_ids_without_extension:
    file_path = TRAIN_WAV_DIR / f"{train_wav_id}.wav"
    if os.path.getsize(file_path) <= 100:
        print(f"{train_wav_id}.wav is empty.")
        nearly_empty_files.append(train_wav_id.split('.')[0])

KeyboardInterrupt: 

In [ ]:
# pydub を用いて、nearly_empty_files の最初 3 個のファイルを再生
from pydub import AudioSegment
from pydub.playback import play

for train_wav_id in nearly_empty_files[:3]:
    file_path = TRAIN_WAV_DIR / f"{train_wav_id}.wav"
    sound = AudioSegment.from_wav(file_path)
    play(sound)

[wav @ 0x145fe4000b80] Cannot check for SPDIF 0KB sq=    0B f=0/0   
Input #0, wav, from '/tmp/tmpicqjsf2q.wav':
  Duration: N/A, bitrate: 512 kb/s
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 32000 Hz, 1 channels, s16, 512 kb/s


[wav @ 0x153478000b80] Cannot check for SPDIF 0KB sq=    0B f=0/0   
Input #0, wav, from '/tmp/tmppc58keyq.wav':
  Duration: N/A, bitrate: 512 kb/s
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 32000 Hz, 1 channels, s16, 512 kb/s


KeyboardInterrupt: 

In [ ]:
# pydub を用いて、nearly_empty_fies の最初 3 個と同じ id の .mp3 ファイルを再生
for train_wav_id in nearly_empty_files[:3]:
    print(train_wav_id)
    file_path = TRAIN_MP3_DIR / f"{train_wav_id}.mp3"
    sound = AudioSegment.from_mp3(file_path)
    play(sound)

0078302500f1


xcb_connection_has_error() returned true
Input #0, wav, from '/tmp/tmp7r7zq6sv.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:05.69, bitrate: 512 kb/s
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 32000 Hz, 1 channels, s16, 512 kb/s
xcb_connection_has_error() returned true



007ee74acd82


xcb_connection_has_error() returned true
Input #0, wav, from '/tmp/tmp0tzpwol7.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:06.30, bitrate: 512 kb/s
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 32000 Hz, 1 channels, s16, 512 kb/s
xcb_connection_has_error() returned true



00f44bb24416


xcb_connection_has_error() returned true
Input #0, wav, from '/tmp/tmp_dq61c0m.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:06.52, bitrate: 512 kb/s
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 32000 Hz, 1 channels, s16, 512 kb/s
xcb_connection_has_error() returned true


In [ ]:
# pydub を用いて、nearly_empty_fies の最初 3 個と同じ id の .mp3 ファイルを再生
for train_id in train_ids[:3]:
    print(train_id)
    file_path = TRAIN_MP3_DIR / f"{train_id}"
    sound = AudioSegment.from_mp3(file_path)
    play(sound)

000005f3362c.mp3


xcb_connection_has_error() returned true
Input #0, wav, from '/tmp/tmpwodd6iy3.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:01.12, bitrate: 512 kb/s
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 32000 Hz, 1 channels, s16, 512 kb/s
xcb_connection_has_error() returned true



00001dddd002.mp3


xcb_connection_has_error() returned true
Input #0, wav, from '/tmp/tmplflj1k6a.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:02.45, bitrate: 512 kb/s
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 32000 Hz, 1 channels, s16, 512 kb/s
xcb_connection_has_error() returned true



00001e0bc131.mp3


xcb_connection_has_error() returned true
Input #0, wav, from '/tmp/tmp4mssn1yd.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:04.72, bitrate: 512 kb/s
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 32000 Hz, 1 channels, s16, 512 kb/s
xcb_connection_has_error() returned true
